In [2]:
import pickle
import pandas as pd

In [5]:
with open('models/lin_reg.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

C:\OLAMIDE\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 0.23.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\OLAMIDE\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 0.23.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [9]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [10]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [12]:
y_pred.mean()

26.4364841523175

In [27]:
year = 2021
month = 2
taxi_type = 'fhv'

In [28]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [29]:
result_df = pd.DataFrame()

In [30]:
result_df['ride_id'] = df['ride_id']

In [31]:
result_df['Prediction'] = y_pred

In [32]:
result_df

,ride_id,Prediction
1,2021/02_1,26.436484
2,2021/02_2,26.436484
3,2021/02_3,26.436484
4,2021/02_4,26.436484
5,2021/02_5,26.436484
...,...,...
1037687,2021/02_1037687,26.436484
1037688,2021/02_1037688,26.436484
1037689,2021/02_1037689,26.436484
1037690,2021/02_1037690,26.436484


In [33]:
output_file = f's3://nyc-duration-prediction-mide/homework/taxi_type={taxi_type}/year={year:04d}/month={month:02d}.parquet'

In [34]:
result_df.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)